In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.discriminant_analysis import StandardScaler
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
%pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [2]:
random_state=123

In [3]:
# Read in the data
df = pd.read_csv('../Dataset/cleaned_hypertension_data.csv')
df

,Class,Level_of_Hemoglobin,Age,BMI,Sex,Smoking,Physical_activity,salt_content_in_the_diet,Level_of_Stress,Chronic_kidney_disease,Adrenal_and_thyroid_disorders,Genetic_Pedigree_Coefficient,alcohol_consumption_per_day
0,1,11.28,34,23,1,0,45961,48071,2,1,1,0.90,336.333333
1,0,9.75,54,33,1,0,26106,25333,3,0,0,0.23,205.000000
2,1,10.79,70,49,0,0,9995,29465,2,1,0,0.91,67.000000
3,0,11.00,71,50,0,0,10635,7439,1,1,0,0.43,242.000000
4,1,14.17,52,19,0,0,15619,49644,2,0,0,0.83,397.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1,10.14,69,26,1,1,26118,47568,3,1,0,0.02,144.000000
1996,1,11.77,24,45,1,1,2572,8063,3,1,1,1.00,299.666667
1997,1,16.91,18,42,0,0,14933,24753,2,1,1,0.22,369.000000
1998,0,11.15,46,45,1,1,18157,15275,3,0,1,0.72,253.000000


In [4]:
from imblearn.under_sampling import ClusterCentroids, TomekLinks
from imblearn.over_sampling import RandomOverSampler

def splitting_data(df, sampling):
    X = df.drop(['Class'], axis=1)
    y = df['Class']

    if sampling == 'none':
        return X, y
    elif sampling == 'SMOTEENN':
        sampler = SMOTEENN(random_state=0)
    elif sampling == 'SMOTE':
        sampler = SMOTE(random_state=0)
    elif sampling == 'under':
        sampler = RandomUnderSampler(random_state=0)
    elif sampling == 'over':
        sampler = RandomOverSampler(random_state=0)
    elif sampling == 'cluster_centroids':
        sampler = ClusterCentroids(random_state=0)
    elif sampling == 'tomek_links':
        sampler = TomekLinks()

    X_resampled, y_resampled = sampler.fit_resample(X, y)
    return X_resampled, y_resampled


In [5]:

def training(X_train, y_train):
    # Create a KNN classifier with 5 neighbors
    LGBM = lgb.LGBMClassifier()
    # Fit the classifier to the data
    LGBM.fit(X_train, y_train)
    return LGBM

In [6]:
def predict(DT, X_test ,y_test):
    # Predict the labels for the training data X
    y_pred = DT.predict(X_test)
    cr=classification_report(y_test, y_pred)
    print(cr)

In [7]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

def optimize_with_grid(X_train, y_train):
    
    # Initialize the LGBMClassifier
    LGBM = lgb.LGBMClassifier()

    # Define the parameter grid
    param_grid = {
        'num_leaves': [31, 50, 70],
        'learning_rate': [0.01, 0.1, 0.5],
        'n_estimators': [50, 100, 200]
    }

    # Initialize GridSearchCV
    LGBM_cv = GridSearchCV(LGBM, param_grid, cv=5)

    # Fit the grid search to the data
    LGBM_cv.fit(X_train, y_train)

    # Best parameters and best score
    best_params = LGBM_cv.best_params_
    best_score = LGBM_cv.best_score_
    best_estimator = LGBM_cv.best_estimator_
    print(best_params)
    print(best_score)

    return best_estimator

<h1> LGBM on original data with optimization </h1>

In [8]:
# using function with no sampling 
X, y= splitting_data(df, 'none')
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
# Scale the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
#check number of observations in each class in the set
print("Number of observations in each class in the training set:")
print(y.value_counts())

Number of observations in each class in the training set:
0    1013
1     987
Name: Class, dtype: int64


In [10]:
LGBM1 = training(X_train, y_train)
y_pred = predict(LGBM1, X_test, y_test)

: 

In [ ]:
best_DT1 = optimize_with_grid(X_train, y_train)
prediction = predict(best_DT1, X_test, y_test)

<h1> DT using SMOTE sampling </h1>

In [ ]:
X,y = splitting_data(df, 'SMOTE')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#check number of observations in each class in the set
print("Number of observations in each class in the training set:")
print(y.value_counts())

In [ ]:
LGBM2 =training(X_train, y_train)
y_pred = predict(LGBM2, X_test, y_test)

In [ ]:
best_LGBM2 = optimize_with_grid(X_train, y_train)
prediction = predict(best_LGBM2, X_test, y_test)

<h1> DT using SMOTEENN sampling </h1>

In [ ]:
X,y = splitting_data(df, 'SMOTEENN')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
print("Number of observations in each class in the set:")
print(y.value_counts())

In [ ]:
LGBM3 =training(X_train, y_train)
y_pred = predict(LGBM3, X_test, y_test)

In [ ]:
best_LGBM3 = optimize_with_grid(X_train, y_train)
prediction = predict(best_LGBM3, X_test, y_test)

<h1> DT on Random undersampling </h1>

In [ ]:
X,y = splitting_data(df, 'under')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
print("Number of observations in each class in the set:")
print(y.value_counts())

In [ ]:
LGBM4 =training(X_train, y_train)
y_pred = predict(LGBM4, X_test, y_test)

In [ ]:
best_LGBM4 = optimize_with_grid(X_train, y_train)
prediction = predict(best_LGBM4, X_test, y_test)

<h1> DT on Cluster Centroids </h1>

In [ ]:
X,y = splitting_data(df, 'cluster_centroids')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
print("Number of observations in each class in the set:")
print(y.value_counts())

In [ ]:
LGBM5 =training(X_train, y_train)
y_pred = predict(LGBM5, X_test, y_test)

In [ ]:
best_LGBM5 = optimize_with_grid(X_train, y_train)
prediction = predict(best_LGBM5, X_test, y_test)

<h1> DT on Tomek Links </h1>

In [ ]:
X,y = splitting_data(df, 'tomek_links')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
print("Number of observations in each class in the set:")
print(y.value_counts())

In [ ]:
LGBM6 =training(X_train, y_train)
y_pred = predict(LGBM6, X_test, y_test)

In [ ]:
best_LGBM6 = optimize_with_grid(X_train, y_train)
prediction = predict(best_LGBM6, X_test, y_test)